# MultiBand Single Pass (MBSP) Demonstration

This notebook demonstrates a basic implementation of the MBSP algorithm
described in *Varon et al. (2021)* for detecting large methane plumes using
Sentinel‑-2 imagery. The MBSP method compares band‑11 and band‑12
reflectances for a single scene to retrieve methane column enhancements.
We keep core routines in `src/mbsp.py` for reuse, while image selection
and experimentation remain in the notebook.

## MBSP versus MBMP

MBSP uses two spectral bands from a single acquisition. A scaling coefficient
is fitted between bands 11 and 12 over the scene and the fractional absorption
is computed as:

\begin{align}
R_{MBSP} = \frac{c R_{12} - R_{11}}{R_{11}}
\end{align}

where $c$ is the slope from fitting $R_{12}$ to $R_{11}$. This approach relies
on the surface behaving similarly in both bands.

The MultiBand MultiPass (MBMP) technique performs the MBSP retrieval on two
different dates and subtracts them to reduce artifacts. MBMP generally provides
better precision when a good plume‑free reference image is available.

In [ ]:
import datetime as dt

import ee
import geemap

# Authenticate with Earth Engine
ee.Authenticate()
ee.Initialize()


## Helper Functions

In [ ]:
def mask_s2_clouds(image: ee.Image) -> ee.Image:
    """Mask clouds using the QA60 band."""
    qa = image.select("QA60")
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    masked = image.updateMask(mask).divide(10000)
    return masked.copyProperties(image, image.propertyNames())  # <-- keep metadata!


def mbsp_fractional_image(image: ee.Image, region: ee.Geometry) -> ee.Image:
    """
    Compute the MBSP (Multi-Band Single-Pass) fractional methane signal
    for a Sentinel-2 scene following Varon et al. 2021 (Atmos. Meas. Tech.).

    MBSP definition
    ----------------
        R_MBSP = ( c · R12  –  R11 ) / R11

    where
        R11, R12 : TOA reflectances for Sentinel-2 bands 11 (1610 nm) and 12 (2190 nm)
        c        : scene-wide slope from a zero-intercept linear regression of
                   R11 on R12 (c = Σ R11·R12 / Σ R12²)

    R_MBSP isolates the methane-induced absorption in band 12 by:
      1) Bringing band 12 onto the radiometric scale of band 11 (multiplying by c)
      2) Differencing and normalising by R11.

    A *negative* R_MBSP indicates that band 12 is darker than expected and is
    therefore consistent with CH₄ absorption.

    Parameters
    ----------
    image  : ee.Image
        Sentinel-2 L1C/L2A image containing bands ‘B11’ and ‘B12’.
    region : ee.Geometry
        Area (ideally plume-free) used to derive the scene-wide slope *c*.

    Returns
    -------
    ee.Image
        Single-band image named ‘R’ holding the pixel-wise MBSP signal.
        The fitted slope *c* is attached as image property ‘slope’.
    """
    # --- 1. Numerator and denominator for the regression slope -----------------
    # c = Σ( R11 * R12 ) / Σ( R12² )
    num_img = image.select("B11").multiply(image.select("B12"))  # R11·R12
    den_img = image.select("B12").multiply(image.select("B12"))  # R12²

    num_sum = num_img.reduceRegion(
        reducer=ee.Reducer.sum(), geometry=region, scale=20, bestEffort=True
    )
    den_sum = den_img.reduceRegion(
        reducer=ee.Reducer.sum(), geometry=region, scale=20, bestEffort=True
    )

    # Convert to ee.Number for further arithmetic
    slope = ee.Number(num_sum.get("B11")).divide(ee.Number(den_sum.get("B12")))

    # --- 2. Per-pixel MBSP field ----------------------------------------------
    mbsp = (
        image.select("B12")
        .multiply(slope)  # c · R12
        .subtract(image.select("B11"))
        .divide(image.select("B11"))
        .rename("R")
    )

    # Embed the slope for traceability
    return mbsp.set({"slope": slope})


## Image Selection

In [ ]:
# Location and date range of interest
lat, lon = 31.6585, 5.9053  # Hassi Messaoud example
start = dt.date(2019, 10, 1)
end = dt.date(2019, 10, 15)

point = ee.Geometry.Point(lon, lat)
collection = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterDate(str(start), str(end))
    .filterBounds(point)
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
    .sort("system:time_start")
    .map(mask_s2_clouds)
)

images = collection.toList(collection.size())
count = images.size().getInfo()
print(f"Found {count} images")

In [ ]:
if count:
    region = point.buffer(1000).bounds()
    m = geemap.Map(center=(lat, lon), zoom=12)
    for i in range(count):
        img = ee.Image(images.get(i))
        date = ee.Date(img.get("system:time_start")).format("YYYY-MM-dd").getInfo()
        r_img = mbsp_fractional_image(img, region)
        m.addLayer(
            r_img,
            {"min": -0.05, "max": 0.05, "palette": ["blue", "white", "red"]},
            f"{date} fractional",
            True,
        )
        rgb = img.select(["B4", "B3", "B2"])
        m.addLayer(rgb, {"min": 0, "max": 0.3}, f"{date} RGB", False)

    styled_pt = ee.FeatureCollection([point]).style(
        **{
            "color": "green",  # outline & fill
            "fillColor": None,  # no fill
            "pointSize": 8,  # pixel radius of the dot
        }
    )
    m.addLayer(styled_pt, {}, "centre pt", True)

In [ ]:
m

The map above contains fractional signal layers for each scene interleaved
with true-color imagery. In the second half we apply the per-pixel water-vapour correction and robust MBSP retrieval described earlier to get CH₄ enhancement maps from a single image.

In [ ]:
import ee
import geemap
import xarray as xr

# LUTs holding k_{band, gas} coefficients for H2O absorption ------------------
lutA = xr.open_dataset("../data/lookup/k_S2A_v1.nc")
lutB = xr.open_dataset("../data/lookup/k_S2B_v1.nc")


# -----------------------------------------------------------------------------
# HELPER FUNCTIONS -------------------------------------------------------------
# -----------------------------------------------------------------------------


def k_lookup(
    ds: xr.Dataset, band: str, gas: str, sza: float, vza: float, pres: float
) -> float:
    """Return per‑band absorption coefficient k_{band, gas} (scalar)."""
    return float(
        ds.k_prime.sel(band=band, gas=gas).interp(sza=sza, vza=vza, pres=pres).values
    )


# -----------------------------------------------------------------------------
# CLOUD MASK ------------------------------------------------------------------
# -----------------------------------------------------------------------------


def mask_s2_clouds(img: ee.Image) -> ee.Image:
    qa = img.select("QA60")
    cloud = 1 << 10
    cirrus = 1 << 11
    m = qa.bitwiseAnd(cloud).eq(0).And(qa.bitwiseAnd(cirrus).eq(0))
    return img.updateMask(m).divide(10000).copyProperties(img, img.propertyNames())


# -----------------------------------------------------------------------------
# CORE – MBSP WITH PER‑PIXEL H2O CORRECTION ----------------------------------
# -----------------------------------------------------------------------------


def mbsp_fractional_image_b9(img: ee.Image, region: ee.Geometry) -> ee.Image:
    """Return an ee.Image containing water‑vapour‑corrected MBSP signal."""

    # 1. Scene geometry --------------------------------------------------------
    sza = img.getNumber("MEAN_SOLAR_ZENITH_ANGLE").getInfo()
    # Sentinel‑2 L1C lacks reliable per‑pixel VZA; assume nadir
    vza = 0.0

    # Pick correct LUT by spacecraft name ------------------------------------
    sat = img.get("SPACECRAFT_NAME").getInfo()
    print(f"sza: {sza}, vza: {vza}, pres: {1013}, sat: {sat}")
    lut = lutA if sat.endswith("2A") else lutB
    k11 = k_lookup(lut, "B11", "H2O", sza, vza, 1013)
    k12 = k_lookup(lut, "B12", "H2O", sza, vza, 1013)
    print(f"k11, k12: {k11}, {k12}")

    # 2. Single‑image PWV retrieval using B9/B8A ------------------------------
    ratio = img.select("B9").divide(img.select("B8A"))
    delta_tau = ratio.log().multiply(-1)  # monochromatic approx.
    # Scale factor α from APDA LUTs ≈ 0.9 mol m‑2 per unit delta_tau
    W = delta_tau.multiply(0.9).rename("PWV")

    # 3. H2O optical depth maps ----------------------------------------------
    tau11 = W.multiply(k11).rename("tau11")
    tau12 = W.multiply(k12).rename("tau12")

    # scene means
    means = tau11.addBands(tau12).reduceRegion(
        ee.Reducer.mean(), region, 20, bestEffort=True
    )
    t11_bar = ee.Number(means.get("tau11"))
    t12_bar = ee.Number(means.get("tau12"))

    # 4. De‑water‑vapour reflectances ----------------------------------------
    r11_dry = img.select("B11").multiply((tau11.subtract(t11_bar)).exp())
    r12_dry = img.select("B12").multiply((tau12.subtract(t12_bar)).exp())

    # 5. Robust slope ---------------------------------------------------------
    num = r11_dry.multiply(r12_dry)
    den = r12_dry.multiply(r12_dry)
    num_sum = num.reduceRegion(ee.Reducer.sum(), region, 20, bestEffort=True)
    den_sum = den.reduceRegion(ee.Reducer.sum(), region, 20, bestEffort=True)
    c = ee.Number(num_sum.values().get(0)).divide(ee.Number(den_sum.values().get(0)))

    # 6. MBSP field -----------------------------------------------------------
    mbsp = r12_dry.multiply(c).subtract(r11_dry).divide(r11_dry).rename("R")
    return mbsp.set({"slope": c, "k11": k11, "k12": k12})


In [ ]:
import datetime as dt

# location + period -------------------------------------------------------
lat, lon = 31.6585, 5.9053
start = dt.date(2019, 10, 10)
end = dt.date(2019, 10, 15)

pt = ee.Geometry.Point(lon, lat)
coll = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterDate(str(start), str(end))
    .filterBounds(pt)
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
    .sort("system:time_start")
    .map(mask_s2_clouds)
)

imgs = coll.toList(coll.size())
n = imgs.size().getInfo()
print(f"{n} usable images\n")

if n:
    reg = pt.buffer(1000).bounds()
    m = geemap.Map(center=(lat, lon), zoom=12)
    for i in range(n):
        im = ee.Image(imgs.get(i))
        date = ee.Date(im.get("system:time_start")).format("YYYY‑MM‑dd").getInfo()
        r_b9 = mbsp_fractional_image_b9(im, reg)
        m.addLayer(
            r_b9,
            {"min": -0.05, "max": 0.05, "palette": ["red", "white", "blue"]},
            f"{date} MBSP‑H2O",
            True,
        )
        r_paper = mbsp_fractional_image(im, reg)
        m.addLayer(
            r_paper,
            {"min": -0.05, "max": 0.05, "palette": ["red", "white", "blue"]},
            f"{date} fractional",
            True,
        )
        # m.addLayer(
        #     im.select(["B4", "B3", "B2"]),
        #     {"min": 0, "max": 0.3},
        #     f"{date} RGB",
        #     False,
        # )
    m.addLayer(
        ee.FeatureCollection([pt]).style(color="00FF00", pointSize=8), {}, "centre"
    )


In [ ]:
m